In [76]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import censusdata
import numpy as np
from census import Census
import us
from ipywidgets import interact, interact_manual, Dropdown, IntSlider
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
import json
import panel as pn
import panel.widgets as pnw
from bokeh.palettes import brewer
from bokeh.plotting import figure

In [2]:
path = r'C:/Users/ShrekTheOger/Documents/GitHub/Data-II-Project'
# path = r'/Users/bowenli/Documents/GitHub/Data-II-Project'
#path = r'C:\Users\engel\Documents\GitHub\Data-II-Project'

In [3]:
df = os.path.join(path+'/refined_data', 'df_income_pop.csv')
df = pd.read_csv(df)
uni_df = os.path.join(path+'/refined_data', 'uni_fund_df.csv')
uni_df = pd.read_csv(uni_df)

In [4]:
#Combining state_id with county_id into same format as in university shape file
l = []
for i in range(len(df['State_id'])):
    if len(str(df['State_id'][i])) == 1:
        if len(str(df['County_id'][i])) == 1:
            a = '0'+ str(df['State_id'][i]) + '00' + str(df['County_id'][i])
        elif len(str(df['County_id'][i])) == 2:
            a = '0'+ str(df['State_id'][i]) + '0' + str(df['County_id'][i])
        else:
            a = '0'+ str(df['State_id'][i]) + str(df['County_id'][i])
    else:
        if len(str(df['County_id'][i])) == 1:
            a = str(df['State_id'][i]) + '00' + str(df['County_id'][i])
        elif len(str(df['County_id'][i])) == 2:
            a = str(df['State_id'][i]) + '0' + str(df['County_id'][i])
        else:
            a = str(df['State_id'][i]) + str(df['County_id'][i])
    l.append(a)

In [5]:
df.insert(3, 'GEOID', l)
df.drop(labels=['State_id', 'County_id'], axis=1, inplace = True)

In [6]:
df

,Year,State,County,GEOID,COUNTYFIPS,Total_population,Total_native,Total_born_in_state,Total_born_out_state,Total_born_outside_us,Total_foreign_born,Income_past12m
0,2010,California,Merced County,06047,6047,250699,188498,156434,29768,2296,62201,18041.0
1,2010,California,Modoc County,06049,6049,9605,8971,6067,2882,22,634,20536.0
2,2010,California,Mono County,06051,6051,13905,11341,7059,4108,174,2564,27321.0
3,2010,California,Monterey County,06053,6053,407435,284647,208218,69900,6529,122788,25776.0
4,2010,California,Napa County,06055,6055,134051,103891,73787,27759,2345,30160,34310.0
...,...,...,...,...,...,...,...,...,...,...,...,...
31069,2019,Tennessee,Crockett County,47033,47033,14399,13774,11231,2490,53,625,23771.0
31070,2019,Tennessee,Lake County,47095,47095,7401,7307,5436,1839,32,94,15732.0
31071,2019,Tennessee,Knox County,47093,47093,461104,438362,272843,161946,3573,22742,33229.0
31072,2019,Washington,Benton County,53005,53005,197518,176385,104182,70549,1654,21133,32882.0


# Retrive Shape file

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
county_shp = os.path.join(path+'/raw_data', 'cb_2020_us_county_20m', 'cb_2020_us_county_20m.shp')
county = gpd.read_file(county_shp)

In [9]:
county['GEOID'] = county['GEOID'].astype(float)

In [10]:
#merge county level income and pop data with shape file county level
df_shape = county.merge(df, how = 'inner', left_on='GEOID', right_on='COUNTYFIPS')

In [11]:
universities_shp = os.path.join(path+'/raw_data', 'Colleges_and_Universities-shp', 'Colleges_and_Universities.shp')
universities = gpd.read_file(universities_shp)

In [12]:
#merge university fund data with shape file university geometries
universities['IPEDSID'] = universities['IPEDSID'].astype(str).astype(int)
uni_shape = universities.merge(uni_df, how = 'inner', on = 'IPEDSID')

In [24]:
def get_geodatasource(gdf):
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson= json_data)

In [60]:
def county_uni_map(state):
    """Generate two maps -- state geographic map and university geographic map based on 
        state selection"""
    
    county_map = df_shape[(df_shape['STATE_NAME'] == state)]
    uni_map = uni_shape[uni_shape['State'] == state]
    return county_map, uni_map

In [78]:
def bokeh_plot_map(gdf, column='Income_past12m', title=''):
    """Plot bokeh map from GeoJSONDataSource """
    
    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset'
    p = figure(title = title, plot_height=400 , plot_width=850, toolbar_location='right', tools=tools)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    return p

In [90]:
# Define a sequential multi-hue color palette
palette = brewer['OrRd'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

vals = a['Income_past12m']
color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')
geosource = get_geodatasource(a)
tools = 'wheel_zoom,pan,reset'
p = figure(title = '', plot_height=640 , plot_width=850, toolbar_location='right', tools=tools)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
    #Add patch renderer to figure
p.patches('xs','ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
          fill_color={'field' :'Income_past12m' , 'transform': color_mapper})
    #Specify figure layout.
p.add_layout(color_bar, 'below')
p

Figure(id='1231', ...)

In [91]:
map_pane = pn.pane.Bokeh(width=400)
map_pane.object = p
app = pn.Column(map_pane)
app

Column
    [0] Bokeh(Figure, width=400)

In [72]:
def map_dash():
    
    from bokeh.models.widgets import DataTable
    map_pane = pn.pane.Bokeh(width = 800)
    state = pnw.Select(name = 'State', options = list(uni_shape['State'].unique()))
    year = pnw.IntSlider(start = 2010, end = 2019, value = 2010)
    #attri = pnw.Select(name = 'Attributes', options = ['Personal Income per Capita','Population Growth'])
    def update_map(event):
        county, uni = county_uni_map(state.value)
        map_lane.object = bokeh_plot_map(county,state)
        return
    year.param.watch(update_map,'value')
    year.param.trigger('value')
    state.param.watch(update_map,'value')
    attri.param.watch(update_map, 'value')
    app = pn.Column(pn.Row(state,year),map_pane)
    return app
        

In [73]:
app = map_dash()

KeyError: Select(name='State', options=['Georgia', 'New Jersey', ...], value='Georgia')

In [61]:
a,b = county_uni_map('Illinois')

In [89]:
a = a[a['Year'] ==2010]
a.T

,20,110,1000,1300,2220,2230,3170,3880,3890,4480,...,27275,28255,28425,29105,29125,29505,30015,30025,30035,30245
STATEFP,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
COUNTYFP,177,155,159,095,029,147,063,047,061,121,...,001,091,195,139,023,059,043,065,067,035
COUNTYNS,01785076,00424279,00424281,00424249,00424216,00424275,00424233,00424225,00424232,00424259,...,00424202,00424247,01785167,01784885,00424213,00424231,00422191,00424234,00424235,00424219
AFFGEOID,0500000US17177,0500000US17155,0500000US17159,0500000US17095,0500000US17029,0500000US17147,0500000US17063,0500000US17047,0500000US17061,0500000US17121,...,0500000US17001,0500000US17091,0500000US17195,0500000US17139,0500000US17023,0500000US17059,0500000US17043,0500000US17065,0500000US17067,0500000US17035
GEOID_x,17177.0,17155.0,17159.0,17095.0,17029.0,17147.0,17063.0,17047.0,17061.0,17121.0,...,17001.0,17091.0,17195.0,17139.0,17023.0,17059.0,17043.0,17065.0,17067.0,17035.0
NAME,Stephenson,Putnam,Richland,Knox,Coles,Piatt,Grundy,Edwards,Greene,Marion,...,Adams,Kankakee,Whiteside,Moultrie,Clark,Gallatin,DuPage,Hamilton,Hancock,Cumberland
NAMELSAD,Stephenson County,Putnam County,Richland County,Knox County,Coles County,Piatt County,Grundy County,Edwards County,Greene County,Marion County,...,Adams County,Kankakee County,Whiteside County,Moultrie County,Clark County,Gallatin County,DuPage County,Hamilton County,Hancock County,Cumberland County
STUSPS,IL,IL,IL,IL,IL,IL,IL,IL,IL,IL,...,IL,IL,IL,IL,IL,IL,IL,IL,IL,IL
STATE_NAME,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,...,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois,Illinois
LSAD,06,06,06,06,06,06,06,06,06,06,...,06,06,06,06,06,06,06,06,06,06


In [70]:
a['Income_past12m']

20       22608.0
21       23413.0
22       23695.0
23       23350.0
24       23686.0
          ...   
30250    22596.0
30251    22786.0
30252    24956.0
30253    25622.0
30254    27812.0
Name: Income_past12m, Length: 1020, dtype: float64

In [66]:
get_geodatasource(a)

AttributeError: unexpected attribute 'show' to GeoJSONDataSource, possible attributes are geojson, js_event_callbacks, js_property_callbacks, name, selected, selection_policy, subscribed_events, syncable or tags